# Phyling API

To use this notebook, you need to create a `.env` file with:

```env
PHYLING_API_KEY=your_api_key

# Server API:   https://api.app.phyling.fr
# Maxi-Hub API: 192.168.1.42:5001
PHYLING_API_URL="https://api.app.phyling.fr"
```

In [ ]:
import json
import time
import logging
import os
from dotenv import load_dotenv

from phyling.api import PhylingAPI
from phyling.api import PhylingRealtime

load_dotenv()
logging.getLogger().setLevel(logging.INFO)

# API connection

In [ ]:
# Create API and Realtime instances
api = PhylingAPI(
    api_key=os.getenv("PHYLING_API_KEY"),
    url=os.getenv("PHYLING_API_URL", "https://api.app.phyling.fr/")
)

realtime = PhylingRealtime(api)

In [ ]:
# Start auto-updating device list every socketIO update
realtime.autoUpdateDeviceList(True)

In [ ]:
# Print the device list (will update automatically in the background)
print(realtime.getDeviceList())

In [ ]:
# Enable auto-updates for device status, indicator, and data for the first connected device
number = realtime.getDeviceList()[0]["number"]
realtime.autoUpdateDeviceStatus(number=number, enabled=True)
realtime.autoUpdateDeviceIndicator(number=number, enabled=True)
realtime.autoUpdateDeviceData(number=number, enabled=True)

In [ ]:
# Print the device status (will update automatically in the background)
print(realtime.getDeviceStatus(number=number))

In [ ]:
# Print the device state only
print(realtime.getDeviceStatus(number=number)["state"])

In [ ]:
# Print the available RPC methods
print(realtime.getDeviceStatus(number=number)["features"])

In [ ]:
# Perform a RPC request
realtime.executeRPC(
    number=number,
    method="v1.record.rec.start",
    params={}
)

In [ ]:
# Print the device indicator (will update automatically in the background - only on record/scenario states)
print(realtime.getDeviceIndicator(number=number))

In [ ]:
# Print the latest realtime data (will update automatically in the background - only on record/scenario states)
lastRtData = realtime.getDeviceData(number=number)
print(lastRtData)

In [ ]:
# Continuously print new realtime data as it arrives
ONLY_FIRST = True
lastRecTime = 0
while True:
    status = realtime.getDeviceStatus(number=number)
    rtData = realtime.getDeviceData(number=number)
    now = time.time()
    if rtData["recTime"] <= lastRecTime:
        continue
    lastRecTime = rtData["recTime"]
    dataStr = f"[{rtData['recTime']:6.3f}] "
    for modname, modcontent in rtData.get("data", {}).items():
        latency = now - status['startRecTimeSinceEpoch'] - rtData["data"][modname]["T"][-1]
        dataStr +=  f" | latency: {int(latency * 1000):4d}ms"
        for key, value in modcontent.items():
            if key in ["T"]:
                continue
            if len(dataStr) > 0:
                dataStr += " | "
            dataStr += f"{modname}.{key}: {value[-1]:.2f}"
            if ONLY_FIRST:
                break
    if len(dataStr) > 0:
        print(dataStr)
    time.sleep(0.01)

In [ ]:
# Continuously print new realtime indicators as they arrive
# {'number': 10200300, 'recTime': 178.123, 'indicators': {'acc_x': {'x': 177.842491, 'y': 0.2968113739043474}}}
ONLY_FIRST = False
lastRecTime = 0
while True:
    status = realtime.getDeviceStatus(number=number)
    rtIndicators = realtime.getDeviceIndicator(number=number)
    now = time.time()
    if rtIndicators["recTime"] <= lastRecTime:
        continue
    lastRecTime = rtIndicators["recTime"]
    dataStr = f"[{rtIndicators['recTime']:6.3f}] "
    for indname, indcontent in rtIndicators.get("indicators", {}).items():
        latency = now - status['startRecTimeSinceEpoch'] - rtIndicators["indicators"][indname]["x"]
        dataStr +=  f"latency: {int(latency * 1000):4d}ms {indname}: {indcontent['y']:.2f}"
        if ONLY_FIRST:
            break
    if len(dataStr) > 0:
        print(dataStr)
    time.sleep(0.01)